In [82]:
import os, os.path

import whoosh
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer
from whoosh import index
import pandas as pd
import shutil
import re
from tqdm import tqdm

In [83]:
df = pd.read_csv('../data/elixier_physics_all.csv', index_col=0)

In [84]:
# Description analysis
# nonna = df[df.description.notna()]
# actual_ds = nonna.description.apply(lambda x: x.split(';')[0]) 
# word_count = actual_ds.apply(lambda x: len(x.split(' ')))
# nonna[word_count < 10][['title', 'description']].apply(lambda x: x['title'] + ' : ' + x['description'], axis=1).values

In [85]:
def parse_description(desc):
    if not pd.isna(desc):
        return desc.split(';')[0]
    return desc
df['description'] = df.description.apply(parse_description)

In [86]:
def split_kws(kw):
    try:
        if pd.isna(kw):
            return kw
        else:
            cleaned = re.sub(' ?Sekundarstufe II?', '', kw).replace(';', '')
            phrases = []
            if 'Deutsch als Zweitsprache' in cleaned:
                phrases.append('Deutsch als Zweitsprache')
                cleaned = cleaned.replace('Deutsch als Zweitsprache', '')
            if 'Deutsch als Fremdsprache' in cleaned:
                phrases.append('Deutsch als Fremdsprache')
                cleaned = cleaned.replace('Deutsch als Fremdsprache', '')
            return ','.join([p for p in cleaned.split(' ') if p] + phrases)
    except Exception as e:
        print(kw)
        raise e

df['schlagwörter'] = df['schlagwörter'].apply(split_kws)

In [87]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer

In [88]:
schema = Schema(url=ID(stored=True),
                title=TEXT(stored=True),
                description=TEXT(stored=True),
                keywords=KEYWORD(stored=True, 
                                 lowercase=True, 
                                 scorable=True, 
                                 commas=True))

In [89]:
from whoosh import index

if os.path.exists("indexdir"):
    shutil.rmtree('indexdir')
    os.mkdir("indexdir")
else:
    os.mkdir("indexdir")

ix = index.create_in("indexdir", schema)

In [90]:
writer = ix.writer()

In [91]:
def handle_nan(value):
    if pd.isna(value):
        return None
    return value

In [92]:
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    writer.add_document(
        url=handle_nan(row['url']), 
        title=handle_nan(row['title']),
        description=handle_nan(row['description']), 
        keywords=handle_nan(row['schlagwörter']))
writer.commit()

100%|███████████████████████████| 2899/2899 [00:01<00:00, 2610.23it/s]


In [93]:
searcher=ix.searcher()

In [95]:
from whoosh.qparser import QueryParser

qp = QueryParser("description", schema=ix.schema)
q = qp.parse(u"augen")

with ix.searcher() as s:
    results = s.search(q)
    print(results[0])
    

<Hit {'description': 'Die Beobachtung der Saturnringe mit eigenen Augen hinterlässt einen bleiben Eindruck. Auch der außergewöhnliche Mond Titan kann mit einfachen Mitteln beobachtet werden.', 'keywords': 'Astronomie,Planet,Beobachtung', 'title': 'Hinweise zur Saturnbeobachtung', 'url': 'http://www.lehrer-online.de/saturn.php'}>


In [96]:
from whoosh.qparser import MultifieldParser

mparser = MultifieldParser(["title", "description", "keywords"], 
                           schema=ix.schema)
q = mparser.parse('augen')

with ix.searcher() as s:
    results = s.search(q)
    print(results[0])

<Hit {'description': 'Satelliten liefern uns wichtige Informationen über das Klima und andere Umweltfragen. Sie untersuchen den "Gesundheitszustand" unseres Planeten Erde mit globaler Perspektive. Hier finden Schüler und Lehrer interessante Unterrichtsmaterialien zur Fernerkundung.', 'keywords': 'Fernerkundung,Satellit,Satellitenbild,Klimaforschung,Wettersatellit,Sensor,Wetterbeobachtung', 'title': 'Mit Satelliten-Augen', 'url': 'https://www.dlr.de/next/desktopdefault.aspx/tabid-7173/11936_read-28173/'}>


In [103]:
with ix.searcher() as s:
    docnums = s.document_numbers(description='dichte')
    keywords = [keyword for keyword, score
                in s.key_terms(docnums, "description")]
    print(keywords)

['dichte', 'auftriebs', 'lkw', 'modell', 'ein']
